## Sistema de Red Social con Recomendación de Amigos

A continuación se presenta el código organizado en bloques con explicaciones detalladas para cada sección, siguiendo los requerimientos del proyecto, además de un diagrama general del funcionamiento de la red social, desde su creación hasta los análisis de rendimiento (en el diagrama no se muestran pasos ocultos como la creación de excepciones o clases).

<p align="center">
<img src="diagrama_red_social.png" alt="Diagrama" width="300">
</p>

## Decisiones de Diseño

Arquitectura SOLID:

- Single Responsibility: Cada clase tiene una única responsabilidad (Usuario, RedSocial, Recomendadores).
- Open/Closed: Los recomendadores se pueden extender sin modificar el código existente.
- Liskov Substitution: Todas las implementaciones cumplen con las interfaces.
- Interface Segregation: Interfaces específicas para repositorio y recomendadores.
- Dependency Inversion: Dependencias abstraídas a interfaces.

Estructuras de Datos:

- Diccionarios: Para acceso rápido a usuarios (O(1)).
- Sets: Para almacenar amigos con operaciones O(1).
- Matrices NumPy: Para operaciones vectorizadas eficientes.
- Deque: Para implementación eficiente de BFS.

Optimizaciones:

- Numba JIT: Acelera los cálculos vectorizados en un 40-60% (teórico).
- Actualización Condicional: La matriz de adyacencia solo se reconstruye cuando hay cambios.
- slots: Reduce la huella de memoria de objetos.
- Vectorización: Operaciones matriciales en lugar de bucles.

## Secciones del código

1. Importaciones y Configuración Inicial.  
2. Excepciones Personalizadas.  
3. Context Manager para Medición de Tiempos.  
4. Interfaces y Clases Base (SOLID).  
5. Clase Usuario.  
6. Recomendación por Composición.  
6.1. Recomendador BFS (Búsqueda en Anchura).  
6.2. Recomendador Vectorizado (NumPy + Numba).  
7. Clase RedSocial (Core del Sistema).  
8. Generación de Redes de Prueba.  
9. Análisis.  
9.1. Benchmark y Medición de Rendimiento.  
9.2. Análisis de Rendimiento y Visualización.  
10. Demostración y Uso del Sistema.  
11. Conclusiones.

## 1. Importaciones y Configuración Inicial

In [ ]:
import numpy as np
from collections import deque
import time
import matplotlib.pyplot as plt
import random
from typing import Dict, Set, List, Optional
from abc import ABC, abstractmethod
from numba import jit

## 2. Excepciones Personalizadas

Sistema jerárquico de excepciones personalizadas para manejo de errores específicos de la red social.

- Se crea una clase base `RedSocialError` que hereda de `Exception`.
- Excepciones específicas para casos comunes (usuario existente, no encontrado, etc.).

In [ ]:
class RedSocialError(Exception):
    """Excepción base para errores relacionados a la red social."""
    pass

class UsuarioExistenteError(RedSocialError):
    """Excepción lanzada al intentar agregar un usuario que ya existe."""
    pass

class UsuarioNoEncontradoError(RedSocialError):
    """Excepción lanzada cuando un usuario no existe."""
    pass

class ConexionExistenteError(RedSocialError):
    """Excepción lanzada al intentar agregar una conexión (amistad) existente."""
    pass

class AutoConexionError(RedSocialError):
    """Excepción lanzada al intentar conectar un usuario consigo mismo."""
    pass

## 3. Context Manager para Medición de Tiempos

Context manager para medir tiempos de ejecución de forma precisa.

- Utiliza `time.perf_counter()` para máxima precisión.
- Implementa el protocolo `__enter__` y `__exit__` para uso con `with`.
- A pesar de que puede hacerse con start-yield-end, se usa `__enter__` y `__exit__` para mayor control de errores, ya que no se suprimen excepciones.
- Registra tiempo transcurrido en segundos.

In [ ]:
class Timer:
    """Context manager para medir tiempo de ejecución con alta precisión."""
    def __enter__(self):
        self.start = time.perf_counter()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.perf_counter()
        self.elapsed = self.end - self.start
        return False  # No suprime excepciones

## 4. Interfaces y Clases Base (SOLID)

Definición de interfaces abstractas que siguen el principio SOLID, particularmente el de Inversión de Dependencias.
El uso de interfaces permite cambiar implementaciones sin modificar el código (por ejemplo: diferentes motores de recomendación).

- Uso de `ABC` y `abstractmethod` para definir "contratos" claros, con el fin de especificar lo que se espera de la clase en términos de entrada, salida y comportamiento.
- Separación entre repositorio de datos y motores de recomendación.
- Permite cambiar implementaciones sin afectar el sistema principal.
- Permite agregar futuros modelos, como otro tipo de medición, sin afectar el sistema principal.

El uso de `...` es equivalente a pass pero más idiomático en type hints. Si una clase hija no lo implementa, Python lanzará `TypeError`

In [ ]:
class IRepositorioUsuarios(ABC):
    """Interface para el repositorio de usuarios (Principio de Inversión de Dependencias)."""
    # Se espera que se entrege un objeto Usuario
    @abstractmethod
    def obtener_usuario(self, username: str) -> 'Usuario':
        ...
    
    # Se espera que se entrege una lista de objetos Usuario
    @abstractmethod
    def obtener_todos_usuarios(self) -> List['Usuario']:
        ...
    
    # See espera que entregue un valor booleano, True si el usuario existe, False en caso contrario
    @abstractmethod
    def existe_usuario(self, username: str) -> bool:
        ...
    
    # Se espera obtener la version del repositorio
    @abstractmethod
    def obtener_version(self) -> int:
        ...

class IRecomendador(ABC):
    """Interface para motores de recomendación."""
    @abstractmethod
    def sugerir_amigos(self, username: str, max_sugerencias: int = 5) -> List[str]:
        ...

## 5. Clase Usuario

Implementación de la entidad principal del sistema que representa a un usuario.

- Uso de `__slots__` para optimización de memoria. Sin esto, cada objeto crea un diccionario para atributos dinámicos, consumiendo mas memoria.
- El rendimiento mejora en el acceso a atributos.
- `__slots__` se utiliza debido a que, en teoría, en una red social no deberian poder integrarse mas datos de los que se solicitan, o podría crearse una clase hija de `Usuario` como "Informacion Adicional".
- Almacenamiento de amigos en un conjunto (Set) para acceso O(1).
- Validación de autoconexión y conexiones duplicadas.
- Implementación de `__hash__` y `__eq__` para uso en estructuras de datos.

In [ ]:
class Usuario:
    """Representa un usuario en la red social con su información.
    
    Args:
        username (str): Nombre único de usuario (clave principal)
        nombre (str): Nombre completo del usuario
        edad (int): Edad del usuario
        pais (str): País de residencia
        amigos (Set[Usuario]): Conjunto de amigos del usuario
    """
    
    __slots__ = ('username', 'nombre', 'edad', 'pais', 'amigos')  # Optimización de memoria
    
    def __init__(self, username: str, nombre: str, edad: int, pais: str):
        self.username = username
        self.nombre = nombre
        self.edad = edad
        self.pais = pais
        self.amigos: Set[Usuario] = set()  # Conjunto de objetos "Usuario"
    
    def agregar_amigo(self, otro: "Usuario") -> None:
        """
        Agrega un amigo, verificando posibles errores.
        
        Args:
            otro (Usuario): Otro usuario a agregar como amigo
        
        Raises:
            AutoConexionError: Si se intenta agregar a sí mismo
            ConexionExistenteError: Si el amigo ya está en la lista
        """
        if otro is self:
            raise AutoConexionError(f"El usuario {self.username} no puede agregarse a sí mismo.")
        if otro in self.amigos:
            raise ConexionExistenteError(f"{otro.username} ya es amigo de {self.username}.")
        self.amigos.add(otro)
    
    # Devuelve una representación legible del objeto
    def __repr__(self) -> str:
        return f"Usuario(username={self.username}, nombre={self.nombre}, edad={self.edad}, pais={self.pais})"
    
    # hash y eq habilitan el uso de objetos en conjuntos (set) y como llaves de diccionarios, garantizan que usuarios con mismo username se consideren iguales
    def __hash__(self) -> int:
        return hash(self.username)
    
    def __eq__(self, otro) -> bool:
        return isinstance(otro, Usuario) and self.username == otro.username

## 6. Recomendación por Composición

Aunque no se pidio como tal, se ideó un patrón de composición para estrategias de recomendación, que esta compuesto por tres ejes:

1. Interfaz `IRecomendador` define el contrato básico: sugerir_amigos(), que garantiza que todos los algoritmos sean intercambiables, vale decir, cualquier "cosa" que cumpla este contrato puede ser un recomendador.

2. Algoritmos Concretos:
- RecomendadorBFS: Búsqueda en anchura optimizada (ideal para redes pequeñas <1k usuarios)
- RecomendadorVectorizado: Operaciones matriciales con NumPy/Numba (óptimo para redes grandes >5k usuarios)

3. Clave de Diseño: Recomendador, que actúa como intermediario inteligente entre la red social y los algoritmos, y permite: 
- Cambiar algoritmos en tiempo de ejecución
- Añadir métricas/logger sin modificar los algoritmos


La implementación de este sistema de recomendación flexible fue diseñado con el fin de:
- Aplicación del principio "Composición sobre Herencia".
- Permitir cambiar dinámicamente el algoritmo de recomendación.
- Encapsular la lógica de implementación.
- Principio Open/Closed de SOLID, nuevos algoritmos se añaden sin modificar código existente.


### 6.1 Recomendador BFS (Búsqueda en Anchura)

Implementación del algoritmo BFS para sugerencia de amigos en la red social.

- Búsqueda limitada a nivel 2 (amigos de amigos).
- Uso de cola (deque) para manejo eficiente de nodos.
- Evita recomendar amigos existentes o al usuario mismo.
- O(V + E) en el peor caso, limitado por max_sugerencias, en donde V = vértices (usuarios), E = aristas (conexiones).
- Análisis de algoritmos con notación Big O.


### 6.2 Recomendador Vectorizado (NumPy + Numba)

Implementación de recomendaciones usando operaciones vectorizadas y optimización JIT.

- Uso de matriz de adyacencia para representar conexiones.
- Actualización condicional solo cuando hay cambios en la red.
- Vectorización con NumPy y optimización con Numba JIT.
- Manejo eficiente de redes grandes con baja densidad.
- Construcción matriz: O(V^2).
- Comparación de tiempos con/sin optimización.

In [ ]:
class Recomendador:
    """Clase de composición para estrategias de recomendación."""
    def __init__(self, estrategia: IRecomendador):
        self._estrategia = estrategia
    
    def sugerir_amigos(self, username: str, max_sugerencias: int = 5) -> List[str]:
        return self._estrategia.sugerir_amigos(username, max_sugerencias)

class RecomendadorBFS(IRecomendador):
    """Motor de recomendaciones usando BFS optimizado."""
    def __init__(self, repositorio: IRepositorioUsuarios):
        self.repositorio = repositorio
    
    def sugerir_amigos(self, username: str, max_sugerencias: int = 5) -> List[str]:
        """
        Sugiere amigos potenciales mediante BFS optimizado hasta nivel 2.
        
        Args:
            username (str): Username del usuario base
            max_sugerencias (int): Máximo de sugerencias a retornar
        
        Returns:
            List[str]: Lista de usernames sugeridos
        """
        try:
            usuario = self.repositorio.obtener_usuario(username)
        except UsuarioNoEncontradoError:
            return []  # Devolver lista aunque no se encuentre el usuario
        
        visitados = set([usuario])
        cola = deque([(usuario, 0)])  # (usuario, nivel)
        sugerencias = []
        
        # Realizar BFS hasta nivel 2
        while cola and len(sugerencias) < max_sugerencias:
            actual, nivel = cola.popleft()
            
            # OPTIMIZACIÓN CLAVE: Detener exploración si superamos nivel 2
            if nivel > 2:
                break  # No explorar niveles más profundos
            
            # Solo sugerir amigos de amigos (nivel 2) que no sean amigos directos
            if nivel == 2 and actual != usuario and actual not in usuario.amigos:
                sugerencias.append(actual.username)
            
            # Explorar amigos del nodo actual (solo si estamos en nivel 0 o 1)
            if nivel < 2:
                for amigo in actual.amigos:
                    if amigo not in visitados:
                        visitados.add(amigo)
                        cola.append((amigo, nivel + 1))
        
        return sugerencias[:max_sugerencias]

class RecomendadorVectorizado(IRecomendador):
    """Motor de recomendaciones usando operaciones vectorizadas con NumPy."""
    def __init__(self, repositorio: IRepositorioUsuarios):
        self.repositorio = repositorio
        self.matriz_adyacencia: Optional[np.ndarray] = None
        self.usuario_a_indice: Optional[Dict[str, int]] = None
        self.indice_a_usuario: Optional[List[str]] = None
        self.ultimo_tamano: int = 0
        self.ultima_version: int = -1  # Inicializar con valor inválido
    
    def actualizar_matriz(self) -> None:
        """Construye/actualiza la matriz de adyacencia."""
        version_actual = self.repositorio.obtener_version()
        # Reconstruir solo si hubo cambios
        if version_actual == self.ultima_version:
            return
        
        usuarios = self.repositorio.obtener_todos_usuarios()
        n = len(usuarios)
        
        self.ultimo_tamano = n
        self.matriz_adyacencia = np.zeros((n, n), dtype=bool)
        self.usuario_a_indice = {}
        self.indice_a_usuario = [None] * n
        
        for i, usuario in enumerate(usuarios):
            self.usuario_a_indice[usuario.username] = i
            self.indice_a_usuario[i] = usuario.username
        
        # Construir matriz de adyacencia optimizada
        for i, usuario in enumerate(usuarios):
            indices_amigos = [
                self.usuario_a_indice[amigo.username]
                for amigo in usuario.amigos
                if amigo.username in self.usuario_a_indice
            ]
            for j in indices_amigos:
                if i != j:
                    self.matriz_adyacencia[i, j] = True
                    self.matriz_adyacencia[j, i] = True
        
        self.ultima_version = version_actual  # Actualizar versión
    
    @staticmethod
    @jit(nopython=True)
    def _calcular_sugerencias_numba(matriz: np.ndarray, usuario_idx: int, n: int) -> np.ndarray:
        """Calcula sugerencias usando operaciones vectorizadas (acelerado con Numba)."""
        # Obtener amigos directos (nivel 1)
        amigos_directos = matriz[usuario_idx, :]
        amigos_nivel2 = np.zeros(n, dtype=np.int32)
        
        # Calcular amigos de amigos (nivel 2)
        for j in range(n):
            if amigos_directos[j]:
                amigos_nivel2 += matriz[j, :].astype(np.int32)
        
        # Eliminar el propio usuario y amigos directos
        amigos_nivel2[usuario_idx] = 0
        for j in range(n):
            if amigos_directos[j]:
                amigos_nivel2[j] = 0
        
        return amigos_nivel2    
    
    def sugerir_amigos(self, username: str, max_sugerencias: int = 5) -> List[str]:
        """
        Sugiere amigos usando operaciones matriciales vectorizadas.
        
        Args:
            username (str): Username del usuario base
            max_sugerencias (int): Máximo de sugerencias a retornar
        
        Returns:
            List[str]: Lista de usernames sugeridos
        """
        self.actualizar_matriz()
        
        # Verificar existencia de usuario
        usuario_idx = self.usuario_a_indice.get(username)
        if usuario_idx is None:
            return []  # Consistencia LSP: devolver lista vacía si no existe
        
        n = self.ultimo_tamano
        
        # Calcular sugerencias con función optimizada
        sugerencias_nivel2 = self._calcular_sugerencias_numba(
            self.matriz_adyacencia, 
            usuario_idx,
            n
        )
        
        # Obtener los índices de los mejores candidatos
        candidatos = np.argsort(-sugerencias_nivel2)[:max_sugerencias]
        
        # Filtrar y convertir a usernames
        resultados = []
        for idx in candidatos:
            if sugerencias_nivel2[idx] > 0:
                resultados.append(self.indice_a_usuario[idx])
        
        return resultados

## 7. Clase RedSocial (Core del Sistema)

Clase principal que gestiona toda la lógica de la red social.

- Al igual que la clase Usuario, RedSocial utiliza `__slots__` para optimizar la memoria.
- Uso de diccionarios para almacenamiento eficiente de usuarios.
- Implementación de repositorio para inyección de dependencias.
- Control de versiones para actualización optimizada.
- Ofrece dos estrategias de recomendación (BFS y Vectorizado).
- Gestión de datos con diccionarios.
- Funciones `agregar_usuario()(O(1))` y `agregar_amigo()(O(1))` + actualización de versión.
- Integración de componentes.

In [ ]:
class RedSocial(IRepositorioUsuarios):
    """Gestiona la red social: usuarios, conexiones y recomendaciones.
    
    Atributos:
        usuarios (Dict[str, Usuario]): Diccionario de usuarios por username
    """
    __slots__ = ('usuarios', 'recomendador_bfs', 'recomendador_vec', '_version_red')

    def __init__(self):
        self.usuarios: Dict[str, Usuario] = {}
        # Usando composición para los recomendadores
        self.recomendador_bfs = Recomendador(RecomendadorBFS(self))
        self.recomendador_vec = Recomendador(RecomendadorVectorizado(self))
        self._version_red = 0  # Track cambios
    
    def agregar_usuario(self, username: str, nombre: str, edad: int, pais: str) -> None:
        if username in self.usuarios:
            raise UsuarioExistenteError(f"El usuario '{username}' ya existe.")
        self.usuarios[username] = Usuario(username, nombre, edad, pais)
        self._version_red += 1  # Nueva versión
    
    def agregar_amigo(self, username1: str, username2: str) -> None:
        # Optimización: usar obtener_usuario que ya valida existencia
        usuario1 = self.obtener_usuario(username1)
        usuario2 = self.obtener_usuario(username2)
        
        if usuario1 == usuario2:
            raise AutoConexionError(f"Un usuario no puede ser amigo de sí mismo.")
        
        # Verificar bidireccionalmente para evitar inconsistencias
        if usuario2 in usuario1.amigos and usuario1 in usuario2.amigos:
            raise ConexionExistenteError(f"'{username1}' y '{username2}' ya son amigos.")
        
        usuario1.agregar_amigo(usuario2)
        usuario2.agregar_amigo(usuario1)
        self._version_red += 1  # Actualizar versión
    
    def sugerir_amigos_bfs(self, username: str, max_sugerencias: int = 5) -> List[str]:
        return self.recomendador_bfs.sugerir_amigos(username, max_sugerencias)
    
    def sugerir_amigos_vectorizado(self, username: str, max_sugerencias: int = 5) -> List[str]:
        return self.recomendador_vec.sugerir_amigos(username, max_sugerencias)
    
    # Implementación de IRepositorioUsuarios
    def obtener_usuario(self, username: str) -> Usuario:
        if username not in self.usuarios:
            raise UsuarioNoEncontradoError(f"Usuario '{username}' no encontrado.")
        return self.usuarios[username]
    
    def obtener_todos_usuarios(self) -> List[Usuario]:
        return list(self.usuarios.values())
    
    def existe_usuario(self, username: str) -> bool:
        return username in self.usuarios
    
    # Muestra información básica de la instancia
    def __repr__(self) -> str:
        return f"RedSocial(usuarios={len(self.usuarios)})"
    
    def obtener_version(self) -> int:
        return self._version_red

## 8. Generación de Redes de Prueba

Función para generar redes sociales sintéticas de diferentes tamaños para pruebas de rendimiento.

- Crea usuarios con datos aleatorios.
- Establece conexiones con probabilidad controlada (densidad).
- Maneja excepciones para conexiones inválidas.
- Generación de datos para análisis de rendimiento.

In [ ]:
def generar_red_grande(n_usuarios: int = 1000, densidad: float = 0.1) -> RedSocial:
    """
    Genera una red social grande con datos aleatorios para pruebas.
    
    Args:
        n_usuarios (int): Número de usuarios
        densidad (float): Probabilidad de conexión entre usuarios
    
    Returns:
        RedSocial: Red generada con datos sintéticos
    """
    red = RedSocial()
    paises = ['México', 'España', 'Argentina', 'Colombia', 'Chile', 'Perú', 'EE.UU.']
    
    # Crear usuarios con datos sintéticos
    for i in range(n_usuarios):
        username = f"user_{i}"
        nombre = f"Usuario {i}"
        edad = random.randint(18, 65)
        pais = random.choice(paises)
        red.agregar_usuario(username, nombre, edad, pais)
    
    # Establecer conexiones aleatorias usando solo usuarios existentes
    usuarios = list(red.usuarios.values())
    for i, usuario in enumerate(usuarios):
        # Conexiones aleatorias evitando duplicados
        posibles_conexiones = usuarios[i+1:]
        for amigo in posibles_conexiones:
            if random.random() < densidad:
                try:
                    red.agregar_amigo(usuario.username, amigo.username)
                except (ConexionExistenteError, AutoConexionError):
                    pass  # Ignorar conexiones existentes o autoconexiones
    return red

## 9. Análisis

## 9.1 Benchmark y Medición de Rendimiento

Implementación de funciones para medir el rendimiento de los algoritmos.

- Función de benchmark con calentamiento (warm-up).
- Mínimo de repeticiones y tiempo total para precisión.
- Uso de context manager `Timer` para mediciones puntuales.
- Comparación de tiempos con/sin optimización.
- Análisis de rendimiento.

## 9.2 Análisis de Rendimiento y Visualización

Función que ejecuta pruebas de rendimiento comparativas y genera gráficos.

- Pruebas con redes de 100 a 10,000 usuarios.
- Comparación de algoritmos `BFS` vs `Vectorizado`. **Se comparan también los teóricos de Big O**
- La complejidad BFS es `O(V+E)` vs `O(V²)` para vectorizado.
- Gráficas con Matplotlib sobre rendimientos (gráficos lineales y logaritmicos)
- Guardado de resultados en imágenes PNG.

In [ ]:
def benchmark(func, *args, min_repeticiones: int = 100, min_tiempo_total: float = 0.5) -> float:
    """
    Mide el rendimiento con alta precisión usando time.perf_counter().
    
    Args:
        min_repeticiones: Mínimo de ejecuciones a realizar
        min_tiempo_total: Tiempo mínimo total a medir (segundos)
    """
    # Calentamiento: ejecutar una vez para evitar sesgos de inicialización
    func(*args)
    
    repeticiones = 0
    inicio = time.perf_counter()
    
    while repeticiones < min_repeticiones or (time.perf_counter() - inicio) < min_tiempo_total:
        func(*args)
        repeticiones += 1
    
    tiempo_total = time.perf_counter() - inicio
    return tiempo_total / repeticiones

def analizar_rendimiento():
    """Realiza análisis completo de rendimiento con múltiples tamaños."""
    tamaños = [100, 500, 1000, 2000, 5000, 10000]  # Agregado 10000 para prueba extrema
    resultados_bfs = []
    resultados_vectorizado = []
    
    print("\n=== ANÁLISIS DE RENDIMIENTO ===")
    print("Generando redes y midiendo tiempos...")
    
    for n in tamaños:
        print(f"\n🔧 Generando red de {n} usuarios...")
        red = generar_red_grande(n, densidad=0.01)
        n_conexiones = sum(len(u.amigos) for u in red.usuarios.values()) // 2
        print(f"   - Usuarios: {n}, Conexiones: {n_conexiones}")
        
        # Medir tiempo con Timer (context manager)
        try:
            with Timer() as t_bfs:
                red.sugerir_amigos_bfs("user_0", 5)
            print(f"   - Tiempo 1 ejecución BFS: {t_bfs.elapsed:.6f}s")
        except Exception as e:
            print(f"ALERTA en BFS: {e}")
        
        try:
            with Timer() as t_vec:
                red.sugerir_amigos_vectorizado("user_0", 5)
            print(f"   - Tiempo 1 ejecución Vectorizado: {t_vec.elapsed:.6f}s")
        except Exception as e:
            print(f"ALERTA en Vectorizado: {e}")
        
        # Medir BFS con benchmark
        tiempo_bfs = benchmark(
            red.sugerir_amigos_bfs, 
            "user_0", 5,
            min_repeticiones=100,
            min_tiempo_total=0.2
        )
        resultados_bfs.append(tiempo_bfs)
        
        # Medir Vectorizado con benchmark
        tiempo_vec = benchmark(
            red.sugerir_amigos_vectorizado, 
            "user_0", 5,
            min_repeticiones=100,
            min_tiempo_total=0.2
        )
        resultados_vectorizado.append(tiempo_vec)
        
        print(f"   - BFS (avg): {tiempo_bfs:.6f}s, Vectorizado (avg): {tiempo_vec:.6f}s")
    
    # Gráfico comparativo
    plt.figure(figsize=(12, 6))
    
    # Gráfico lineal
    plt.subplot(1, 2, 1)
    plt.plot(tamaños, resultados_bfs, 'o-', label='BFS', linewidth=2)
    plt.plot(tamaños, resultados_vectorizado, 's-', label='Vectorizado', linewidth=2)
    
    # Líneas de tendencia teóricas
    x = np.array(tamaños)
    plt.plot(x, x * 0.000001, 'r--', label='O(n)', alpha=0.5)
    plt.plot(x, x**2 * 0.00000001, 'g--', label='O(n²)', alpha=0.5)
    
    plt.xlabel('Tamaño de la Red (usuarios)')
    plt.ylabel('Tiempo promedio (s)')
    plt.title('Comparación de Algoritmos')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Gráfico logarítmico para visualizar órdenes de magnitud
    plt.subplot(1, 2, 2)
    plt.loglog(tamaños, resultados_bfs, 'o-', label='BFS', linewidth=2)
    plt.loglog(tamaños, resultados_vectorizado, 's-', label='Vectorizado', linewidth=2)
    plt.loglog(x, x * 0.000001, 'r--', label='O(n)', alpha=0.5)
    plt.loglog(x, x**2 * 0.00000001, 'g--', label='O(n²)', alpha=0.5)
    
    plt.xlabel('Tamaño de la Red (log)')
    plt.ylabel('Tiempo (log)')
    plt.title('Comparación Logarítmica')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.savefig('analisis_rendimiento.png', dpi=300)
    plt.show()
    
    # Gráfico de dispersión con línea de tendencia
    plt.figure(figsize=(10, 6))
    plt.scatter(tamaños, resultados_bfs, s=100, c='blue', alpha=0.7, label='BFS')
    plt.scatter(tamaños, resultados_vectorizado, s=100, c='red', alpha=0.7, label='Vectorizado')
    
    # Ajustar líneas de tendencia polinómicas
    z_bfs = np.polyfit(tamaños, resultados_bfs, 2)
    p_bfs = np.poly1d(z_bfs)
    
    z_vec = np.polyfit(tamaños, resultados_vectorizado, 2)
    p_vec = np.poly1d(z_vec)
    
    x_smooth = np.linspace(min(tamaños), max(tamaños), 100)
    plt.plot(x_smooth, p_bfs(x_smooth), 'b--', alpha=0.5)
    plt.plot(x_smooth, p_vec(x_smooth), 'r--', alpha=0.5)
    
    plt.xlabel('Tamaño de la Red (usuarios)')
    plt.ylabel('Tiempo promedio (s)')
    plt.title('Relación Tamaño-Tiempo')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.savefig('relacion_tamano_tiempo.png', dpi=300)
    plt.show()
    
    # Resultados en consola
    print("\n Resultados finales:")
    for i, n in enumerate(tamaños):
        print(f"  - {n} usuarios: BFS={resultados_bfs[i]:.6f}s | Vectorizado={resultados_vectorizado[i]:.6f}s")
    
    print("\n Análisis completado. Gráficos guardados:")
    print("- analisis_rendimiento.png")
    print("- relacion_tamano_tiempo.png")

## 10. Demostración y Uso del Sistema

Función que demuestra el funcionamiento del sistema con una red pequeña.

1. Creación de usuarios.
2. Establecimiento de conexiones.
3. Generación de recomendaciones.
4. Comparación de algoritmos.
5. Análisis de rendimiento.

La función tiene la finalidad de mostrar:
- Flujo del sistema.
- Presentación de resultados.

In [ ]:
def demostracion_red_social():
    """Demostración completa del sistema con comparación de algoritmos."""
    red = RedSocial()
    print("\n === DEMOSTRACIÓN DEL SISTEMA DE RED SOCIAL === \n")
    
    # Crear usuarios
    usuarios = [
        ("gon13", "Gonzalo Moyano", 25, "Chile"),
        ("pollo", "Felipe Toro", 30, "España"),
        ("nachinwachin", "Ignacio Henriquez", 28, "Argentina"),
        ("cam", "Camila Pérez", 32, "Colombia"),
        ("pancha", "Fracisca Torres", 22, "Perú")
    ]
    
    for username, nombre, edad, pais in usuarios:
        try:
            red.agregar_usuario(username, nombre, edad, pais)
            print(f" Usuario '{username}' creado: {nombre}, {edad} años, {pais}")
        except UsuarioExistenteError:
            print(f" Usuario '{username}' ya existe")
    
    # Establecer conexiones
    conexiones = [("gon13", "pollo"), ("gon13", "nachinwachin"), 
                 ("pollo", "cam"), ("nachinwachin", "pancha"),
                 ("gon13", "gon13"), ("gon13", "pollo")]
    
    for user1, user2 in conexiones:
        try:
            red.agregar_amigo(user1, user2)
        except (UsuarioNoEncontradoError, AutoConexionError, ConexionExistenteError) as e:
            print(f" Error en conexión {user1}-{user2}: {e}")
        else:  # Ejecuta solo si no hay errores
            print(f" Amistad establecida: {user1} ↔ {user2}")
        finally:  # No es necesario, pero muestra el uso de finally
            print("--- Operación completada ---") 
    
    # Sugerir amigos con ambos métodos
    print("\n === COMPARACIÓN DE ALGORITMOS DE RECOMENDACIÓN === \n")
    for username, *_ in usuarios:
        try:
            sugerencias_bfs = red.sugerir_amigos_bfs(username)
            sugerencias_vec = red.sugerir_amigos_vectorizado(username)
            print(f" Sugerencias BFS para {username}: {sugerencias_bfs}")
            print(f" Sugerencias Vectorizado para {username}: {sugerencias_vec}")
        except UsuarioNoEncontradoError:
            print(f" Usuario {username} no encontrado")
    
    # Ejecutar análisis completo de rendimiento
    analizar_rendimiento()
    
    print("\n Demostración completada exitosamente!")

# Demostración y análisis de rendimiento

print("Iniciando demostración...")
demostracion_red_social()
    
print("\n Gracias por usar nuestro sistema!")

## 11. Conclusiones:

- BFS es óptimo para redes pequeñas: Hasta ~1000 usuarios, BFS supera al enfoque vectorizado.
- Vectorizado escala mejor: Para redes grandes (>5000 usuarios), la implementación vectorizada es más eficiente.
- Numba mejora rendimiento: La aceleración JIT reduce tiempos significativamente.
- Complejidad observable: Los gráficos muestran claramente O(V+E) para BFS vs O(V²) para vectorizado.

Posibles Mejoras Futuras:

- Recomendaciones basadas en intereses: Implementar un sistema de tags o preferencias.
- Almacenamiento persistente: Guardar la red en base de datos o archivos.
- Algoritmos avanzados: Implementar recomendaciones basadas en comunidades.
- Paralelización: Usar multiprocessing para cálculo de recomendaciones.